# Coin Hunter

Some hunters will hunt for the coins in a grid-like world. The hunters have a BDI architecture.



---



In [1]:
import agentpy as ap
import pathfinding as pf        #In case you want to use pathfinding algorithms for the agent's plan
import matplotlib.pyplot as plt
from owlready2 import *
import itertools
import random
import IPython
import math

In [2]:
#DO NOT EXECUTE MORE THAN ONCE, THE ONTOLOGY WILL ACCUMULATE INFORMATION AND LEAD TO ERRORS

#Simple general ontology using Owlready2, of course you can use an ontology from an OWL file

#If you need to execute this cell again you can do it deleting the ontology as follows:
#onto.destroy(update_relation = True, update_is_a = True)

#Ontology name and path
onto = get_ontology("file:///content/coin_onto.owl")

#opening the ontology
with onto:

    #My SuperClass
    class Entity(Thing):
        pass

    class Hunter(Entity):
        pass

    class Coin(Entity):
        pass

    class Place(Thing):
        pass

    #Property to describe the place of an entity in the grid
    class is_in_place(ObjectProperty):
        domain = [Entity]
        range = [Place]
        pass

    #Property that specifies the position of a Place
    class at_position(DataProperty,FunctionalProperty):
        domain = [Place]
        range = [str]
        pass

    #Property to describe how many coins the agent can see
    class coins_within_reach(ObjectProperty):
        domain = [Hunter]
        range = [int]


In [3]:
#THE HUNTER'S AGENT CLASS

class HunterAgent(ap.Agent):

    #BDI related functionality:

    #Agentś "perception" see() function
    def see(self,e):

        """ It gets all coins (agentType 1) that the agent can see """

        #The range of observetion from the Hunter's viewpoint,
        #if you want the Hunter to see all the world at the same time, then use:
        #seeRange = self.model.p.worldSize[0]
        seeRange = 10
        #A list of all neighboors that are coins (not hunters)
        p = [a for a in e.neighbors(self, distance=seeRange) if a.agentType==1]

        return p


    #Belief revision function:
    def brf(self,p):

        """
        It will update the Beilef system of the agent.
        The beleif system is based on the Ontology
        """

        #Destroys previous beliefs
        for coin in self.this_hunter.coins_within_reach:
            destroy_entity(coin.is_in_place[0])
            destroy_entity(coin)
        destroy_entity(self.this_hunter.is_in_place[0])

        #Ontologically instantiate the hunter
        currentPos = self.model.coinWorld.positions[self]
        self.this_hunter.is_in_place = [Place(at_position = str(currentPos))]

        #Ontologically instantiate the coins at reach
        for c in p:
            theCoin = Coin(is_in_place = [Place()])
            theCoin.is_in_place[0].at_position = str(self.model.coinWorld.positions[c])
            self.this_hunter.coins_within_reach.append(theCoin)


    #The options function (where it gets its Desires)
    def options(self):

        """ It returns the available goals to persue.
        These are based on each Coin distance relative to the Hunter
        """
        distances = {}

        #For each Coin at reach from the Hunter
        for onto_coin in self.this_hunter.coins_within_reach:
            coin_pos = eval(onto_coin.is_in_place[0].at_position)
            hunter_pos = eval(self.this_hunter.is_in_place[0].at_position)
            #Calculate the euclidean distance:
            d = math.sqrt((coin_pos[0]-hunter_pos[0])**2 + (coin_pos[1]-hunter_pos[1])**2)
            #Store in a dictionary:
            distances[onto_coin] = d

        return distances


    #The filter function (where it gets the Intention)
    def filter(self):

        """
        This will return the closest coin as the target (Intention).
        It is based on the Hunter's Desires.
        """

        #Sort the dictionary based on each coin's distance:
        desires = {x:y for x,y in sorted(self.D.items(),key=lambda item:item[1])}

        #return the first coin
        if desires:
            return list(desires.items())[0][0]
        else:
            return None


    #The plan function (Where the agent creates a plan)
    def plan(self):

        """
        Here the Hunter will create a plan towards the current Intention.
        This returns a plan in the form of a list of tuples (x,y).
        Each step on the plan is a step over the grid on the main four
        directions (not diagonal).
        Example:
            [(1,0),(0,-1),(-1,0),(1,0),etc.]
        """

        if self.I == None:
            if random.randint(0,1) == 0:
                return [(random.choice([-1,1]),0)]
            elif random.randint(0,1) == 1:
                return [(0,random.choice([-1,1]))]
            else:
                return [(0,0)]

        thePlanX = []
        thePlanY = []

        #get target porition
        coinPos = eval(self.I.is_in_place[0].at_position)
        #get hunter position
        hunterPos = eval(self.this_hunter.is_in_place[0].at_position)
        #calculate distances among each axis
        distance2D = (coinPos[0]-hunterPos[0],coinPos[1]-hunterPos[1])

        #create a list of atomic steps (1 or -1) on the X axis
        for i in range(abs(distance2D[0])):
            val = 1 if distance2D[0] >= 0 else -1
            thePlanX.append(val)

        #create a list of atomic steps (1 or -1) on the Y axis
        for j in range(abs(distance2D[1])):
            val = 1 if distance2D[1] >= 0 else -1
            thePlanY.append(val)

        #creates a list of tuples filling with ceros
        thePlanX = list(zip(thePlanX,[0 for _ in range(len(thePlanX))]))
        thePlanY = list(zip([0 for _ in range(len(thePlanY))],thePlanY))

        #Creates a final list of the whole plan and shuffles it
        #(The shuffling part is to have a less boring agent)
        thePlan = thePlanX + thePlanY
        random.shuffle(thePlan)

        return thePlan


    #The main BDI algorithm
    def BDI(self, p):

        """
        This function calls all functions from the BDI architecture.
        """

        #Calling brf at the beginning
        self.brf(p)

        #If the Hunter reached a goal, then update Desires and Intentions,
        #and create new Plan
        if self.intentionSucceded:
            self.intentionSucceded = False
            self.D = self.options()
            self.I = self.filter()
            self.currentPlan = self.plan()


    #The function to execute actions
    def execute(self):

        """
        This function will execute the plan, action by action.
        Each action is a tuple that has a 1 or -1,
        so they describe if the agent needs to move in one
        direction or the other.
        """

        #If the plan hasn't finished
        if len(self.currentPlan) > 0:
            #Then  get the next action
            currentAction = self.currentPlan.pop()
        else: # If the plan has finished
            #It means the Hunter has succeded in it current task
            self.intentionSucceded = True
            #So, then do nothing, until next new plan
            currentAction = (0,0)

        #Execute the selected action, using move_by()
        self.model.coinWorld.move_by(self,currentAction)


    #Initial beleifs function
    def initBeliefs(self,initPos):

        """
        This function will fill the Belief system, instantiating the first
        concepts form the ontology.
        """

        #initial Place instance
        place = Place(at_position=str(initPos))

        #initial Hunter inistace
        self.this_hunter = Hunter(is_in_place = [place])


    #Initial intentions funtion
    def initIntentions(self):

        """
        This function will provide the first Intention,
        which in this case is empty.
        """

        self.intentionSucceded = True
        self.I = None


    #======================Main Agent's Fcuntions=======================


    #Setup
    def setup(self):

        #HIdentifier of the Hunter Agent
        self.agentType = 0
        self.firstStep = True
        self.currentPlan = []


    #Step
    def step(self):

        #If it is the first step, then late-initialize
        if self.firstStep:
            initPos = self.model.coinWorld.positions[self]
            self.initBeliefs(initPos)
            self.initIntentions()
            self.firstStep = False

        #Execute the main BDI algorithm
        self.BDI(self.see(self.model.coinWorld))

        #Execute next action
        self.execute()

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass

In [4]:
#THE COIN AGENT

#This agent does nothing, it just exists
class CoinAgent(ap.Agent):

    #Setup
    def setup(self):
        #Identifier for Coin Agent
        self.agentType = 1

    #Step
    def step(self):
        pass

    #Update
    def update(self):
        pass

    #End
    def end(self):
        pass

In [5]:
#THE SIMULATION

class CoinWorldModel(ap.Model):

    #A function to get the amount of coins left
    def get_coins(self):
        return len(self.coins)

    #Setup
    def setup(self):

        #Create hunter agents
        self.hunters = ap.AgentList(self,self.p.hunterAgents,HunterAgent)

        #Create coin agents
        self.coins = ap.AgentList(self,self.p.coinAgents,CoinAgent)

        #Create the grid world
        self.coinWorld = ap.Grid(self,self.p.worldSize,track_empty=True)

        #Add agents to the grid
        self.coinWorld.add_agents(self.hunters,random=True,empty=True)
        self.coinWorld.add_agents(self.coins,random=True,empty=True)


    #Step
    def step(self):

        #Do each agents step function
        self.hunters.step()
        self.coins.step()

        #Verify if there is a collition between a Hunter and a Coin
        for hunter in self.hunters:
            for coin in self.coins:
                if coin in self.coinWorld.positions and self.coinWorld.positions[hunter] == self.coinWorld.positions[coin]:
                    #If there is a collition then remove tha Coin from the grid, adn from the simulation
                    self.coinWorld.remove_agents(coin)
                    self.coins.remove(coin)
                    break

        #If there are no coins left, then end the simulation
        if len(self.coins) <= 0:
            self.stop()

    #Update
    def update(self):
        pass

    #End
    def end(self):
        print("Finished!")
        pass

In [6]:
#A FUNCTION TO ANIMATE THEE SIMULATION

def animation_plot(model, ax):
    agent_type_grid = model.coinWorld.attr_grid('agentType')
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Coin Hunt Model \n Time-step: {model.t}, "
                 f"Coins: {model.get_coins()}")

In [8]:
#SIMULATION PARAMETERS

#a random variables (0,1)
r = random.random()

#parameters dict
parameters = {
    "hunterAgents" : 5,     #Amount of Hunters
    "coinAgents" : 50,      #Amount of coins
    "worldSize" : (30,30),      #Grid size
    "steps" : 1000,          #Max steps
    "seed" : 13*r           #seed for random variables (that is random by itself)
}

#============================================================================0

#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = CoinWorldModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())

